<a href="https://colab.research.google.com/github/UranMai/Binding_affinity/blob/master/Morgan_Descriptors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

data = pd.read_csv('compound_1000.csv')
data = data[data.columns[1:3]]
data.head()

,smi,Score
0,NC(=O)c1ccc(-c2ccc3ccccc3n2)cc1,-30.421166
1,Nc1nc2nc(-c3ccccc3)cc(-c3ccccc3)n2n1,-20.839885
2,O=C1[C@@H](N2CCc3cccc(F)c32)CCN1CC(F)(F)F,-30.615744
3,CSCc1cc(C(=O)N2CCc3[nH]ncc3C2)ccc1F,-25.646399
4,CC(=O)Nc1ccc(NC(=O)CSc2ccc(F)cc2)cc1,-29.502308


In [0]:
data['mol'] = data['smi'].apply(lambda x: Chem.MolFromSmiles(x))
data = data.dropna()
data = data.reset_index(drop=True)
data.head()

RDKit ERROR: [21:40:56] SMILES Parse Error: syntax error while parsing: none
RDKit ERROR: [21:40:56] SMILES Parse Error: Failed parsing SMILES 'none' for input: 'none'
RDKit ERROR: [21:40:56] SMILES Parse Error: syntax error while parsing: none
RDKit ERROR: [21:40:56] SMILES Parse Error: Failed parsing SMILES 'none' for input: 'none'


,smi,Score,mol
0,NC(=O)c1ccc(-c2ccc3ccccc3n2)cc1,-30.421166,<rdkit.Chem.rdchem.Mol object at 0x7effe988aa80>
1,Nc1nc2nc(-c3ccccc3)cc(-c3ccccc3)n2n1,-20.839885,<rdkit.Chem.rdchem.Mol object at 0x7effe988aad0>
2,O=C1[C@@H](N2CCc3cccc(F)c32)CCN1CC(F)(F)F,-30.615744,<rdkit.Chem.rdchem.Mol object at 0x7effe988ab20>
3,CSCc1cc(C(=O)N2CCc3[nH]ncc3C2)ccc1F,-25.646399,<rdkit.Chem.rdchem.Mol object at 0x7effe988ab70>
4,CC(=O)Nc1ccc(NC(=O)CSc2ccc(F)cc2)cc1,-29.502308,<rdkit.Chem.rdchem.Mol object at 0x7effe988abc0>


In [0]:
import numpy as np

smiles1 = data.smi[1:1000]
smiles1 = np.array(smiles1)
smiles2 = data.smi[1:1000]
smiles2 = np.array(smiles2)
len(smiles2)

997

# Install libraries


In [0]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local -c rdkit rdkit pytorch

--2020-01-19 21:39:35--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.2.0=py37_0
    - ca-certificates==2019.10.16=0
    - certifi==2019.9.11=py37_0
    - cffi==1.13.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_0
    - conda==4.7.12=py37_0
    - cryptography==2.8=py37h1ba5d50_0
    - idna==2.8=py37_0
    - libedit==3.1.20181209=hc058e9b_0
    -

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs

In [0]:
from argparse import ArgumentParser
import os
import sys

from itertools import product
from typing import List

import math
import numpy as np
from tqdm import tqdm

# 1


In [0]:
'''def morgan_similarity(smiles_1: List[str], smiles_2: List[str], radius: int, sample_rate: float):
    """
    Determines the similarity between the morgan fingerprints of two lists of smiles strings.
    :param smiles_1: A list of smiles strings.
    :param smiles_2: A list of smiles strings.
    :param radius: The radius of the morgan fingerprints.
    :param sample_rate: Rate at which to sample pairs of molecules for Morgan similarity (to reduce time).
    """
    # Compute similarities
    similarities = []
    num_pairs = len(smiles_1) * len(smiles_2) #1000*1000 pairs

    # Sample to improve speed
    if sample_rate < 1.0:
        sample_num_pairs = sample_rate * num_pairs
        sample_size = math.ceil(math.sqrt(sample_num_pairs))
        sample_smiles_1 = np.random.choice(smiles_1, size=sample_size, replace=True)
        sample_smiles_2 = np.random.choice(smiles_2, size=sample_size, replace=True)
    else:
        sample_smiles_1, sample_smiles_2 = smiles_1, smiles_2

    sample_num_pairs = len(sample_smiles_1) * len(sample_smiles_2) #1000*1000 pairs

    for smile_1, smile_2 in tqdm(product(sample_smiles_1, sample_smiles_2), total=sample_num_pairs):
        mol_1, mol_2 = Chem.MolFromSmiles(smile_1), Chem.MolFromSmiles(smile_2)
        fp_1, fp_2 = AllChem.GetMorganFingerprint(mol_1, radius), AllChem.GetMorganFingerprint(mol_2, radius)
        similarity = DataStructs.TanimotoSimilarity(fp_1, fp_2)
        similarities.append(similarity)
    similarities = np.array(similarities)

    # Print results
    print()
    print(f'Average dice similarity = {np.mean(similarities):.4f} +/- {np.std(similarities):.4f}')
    print(f'Minimum dice similarity = {np.min(similarities):.4f}')
    print(f'Maximum dice similarity = {np.max(similarities):.4f}')
    print()
    print('Percentiles for dice similarity')
    print(' | '.join([f'{i}% = {np.percentile(similarities, i):.4f}' for i in range(0, 101, 10)])

    '''

In [0]:
#morgan_similarity(smiles_1=smiles1, smiles_2=smiles2, radius=2, sample_rate=0.9)

In [0]:
from typing import Callable, List, Union

import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
MORGAN_RADIUS = 2
MORGAN_NUM_BITS =2048
Molecule = Union[str, Chem.Mol]



In [0]:
data['features'] = data['mol'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(x, 2, 2048))

In [0]:
import numpy 
numpy.set_printoptions(threshold=sys.maxsize)

In [0]:
def morgan_bin_features(mol): # data which contain mol formats
  features_vector = AllChem.GetMorganFingerprintAsBitVect(mol, 2, 1024)
  features = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(features_vector, features)
  return features

In [0]:
arr = []

for mol in data.mol:
  feat = morgan_bin_features(mol)
  arr.append(feat)


In [0]:
for i in range(len(arr[0])):
  data[i] = [col[i] for col in arr]

In [0]:
data.head()

,smi,Score,mol,fingerprint,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,NC(=O)c1ccc(-c2ccc3ccccc3n2)cc1,-30.421166,<rdkit.Chem.rdchem.Mol object at 0x7effe988aa80>,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Nc1nc2nc(-c3ccccc3)cc(-c3ccccc3)n2n1,-20.839885,<rdkit.Chem.rdchem.Mol object at 0x7effe988aad0>,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,O=C1[C@@H](N2CCc3cccc(F)c32)CCN1CC(F)(F)F,-30.615744,<rdkit.Chem.rdchem.Mol object at 0x7effe988ab20>,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,CSCc1cc(C(=O)N2CCc3[nH]ncc3C2)ccc1F,-25.646399,<rdkit.Chem.rdchem.Mol object at 0x7effe988ab70>,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CC(=O)Nc1ccc(NC(=O)CSc2ccc(F)cc2)cc1,-29.502308,<rdkit.Chem.rdchem.Mol object at 0x7effe988abc0>,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
'''
mol_feat_arr1 = []
for mol in data.mol:
  mol_feat = morgan_bin_features(mol)
  mol_feat_arr1 = numpy.append(mol_feat_arr1, mol_feat)
  mol_feat_arr1 = mol_feat_arr1
mol_feat_arr1
'''